In [160]:
import numpy as np
import pandas as pd
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re

In [161]:
lst=[]
for i in range(2001,2023):
    url = 'https://www.basketball-reference.com/leagues/NBA_'+str(i)+'_totals.html'
    html = urlopen(url)
    soup = BeautifulSoup(html, features="lxml")
    table =soup.select('table')[0]
    rows = table.find_all('tr')
    for row in rows:
        lst.append(row.find_all('a', href=True))

In [163]:
links=[]
names=[]
for i in range(len(lst)):
    try:
        s=str(lst[i][0])
        names.append(re.findall('>([^"]*)<', s))
        links.append(re.findall('"([^"]*)"', s))
    except:
        pass

In [164]:
df=pd.DataFrame(names,columns=['names'])

In [165]:
df['link']=pd.DataFrame(links,columns=['link'])

In [166]:
df.drop_duplicates(inplace=True)

In [167]:
df.reset_index(drop=True,inplace=True)

In [195]:
height=pd.DataFrame(index=df['link'],columns=['height'])
for i in df['link']:
    url = 'https://www.basketball-reference.com'+i
    html = urlopen(url)
    soup = BeautifulSoup(html, features="lxml")
    s=soup.find(string=re.compile("cm,"))
    loc=s.find('cm,')
    height.loc[i,'height']=s[loc-3:loc]

In [202]:
height['height'] = height['height'].apply(pd.to_numeric, errors='coerce')
height.reset_index(drop=True, inplace=True)
df['height']=height['height']

In [204]:
df.to_csv('height.csv',index=False)

In [ ]:
# Other bio stats

In [387]:
other_stats=pd.DataFrame(index=df['link'],columns=['birth_year','college','recruiting_rank','draft','exp'])
for i in df['link']:
    url = 'https://www.basketball-reference.com'+i
    html = urlopen(url)
    soup = BeautifulSoup(html, features="lxml")
    birth_year=soup.find(id='necro-birth')
    birth_year=str(birth_year)
    loc=birth_year.find('data-birth')
    other_stats.loc[i,'birth_year']=birth_year[loc+12:loc+16]


    col=soup.select("a[href*=colleges]")
    try:
        college=re.findall('>([^"]*)<', str(col[1]))[0]
        other_stats.loc[i,'college']=college
    except:
        other_stats.loc[i,'college']='no college'

    s=str(soup.find(id='meta'))
    loc=s.find('Recruiting Rank')
    if loc==-1:
        other_stats.loc[i,'recruiting_rank']='no rank'
    else:
        rec=s[loc:]
        loc=rec.find('(')
        other_stats.loc[i,'recruiting_rank']=rec[loc+1:loc+2]

    loc=s.find(' round')
    if loc==-1:
        other_stats.loc[i,'draft']='undrafted'
    else:
        draft=s[loc:]
        loc=draft.find('overall')
        other_stats.loc[i,'draft']=int(draft[loc-5:loc-3].strip())

   
    loc=s.find('Experience')
    exp=s[loc:]
    loc=exp.find('year')
    if exp[loc-3:loc-1]!='':
        other_stats.loc[i,'exp']=int(exp[loc-3:loc-1])
    else:
        loc=s.find('Career Length')
        exp=s[loc:]
        loc=exp.find('year')
        other_stats.loc[i,'exp']=int(exp[loc-3:loc-1])

In [388]:
other_stats

,birth_year,college,recruiting_rank,draft,exp
link,,,,,
/players/a/abdulma02.html,1969,LSU,no rank,3,9
/players/a/abdulta01.html,1974,San Jose State,no rank,11,6
/players/a/abdursh01.html,1976,California,no rank,3,12
/players/a/alexaco01.html,1973,Virginia,no rank,29,7
/players/a/alexaco02.html,1977,Fresno State,no rank,13,3
...,...,...,...,...,...
/players/w/willizi02.html,2001,Stanford,6,10,1
/players/w/wrighmc01.html,1998,Colorado,no rank,undrafted,1
/players/w/wrighmo01.html,1998,Georgia Tech,no rank,undrafted,1


In [389]:
other_stats.isnull().sum()

birth_year         0
college            0
recruiting_rank    0
draft              0
exp                0
dtype: int64

In [400]:
other_stats=pd.concat([df,other_stats],axis=1)

In [401]:
other_stats.to_csv('nba_other_stats.csv',index=False)

In [2]:
leagues = {1: ['4','Spanish-ACB'],
           2: ['6','Italian-Lega-Basket-Serie-A'],
           3: ['7', 'Turkish-BSL'],
           4: ['8', 'Greek-HEBA-A1'],
           5: ['12', 'French-Jeep-Elite'],
           6: ['18', 'Adriatic-League-Liga-ABA'],
           7: ['35','VTB-United-League']
          }

In [9]:
def get_player_heights(n):
    print(leagues[n][1])
    lst= []
    for i in range(2004,2023):
        for j in range(1,5):
            test=0
            while test==0:
                try:
                    url = 'https://basketball.realgm.com/international/league/'+leagues[n][0]+'/'+leagues[n][1]+'/stats/'+str(i)+'/Totals/All/All/points/All/desc/'+str(j)
                    html = urlopen(url)
                    soup = BeautifulSoup(html, features="lxml")
                    test=1
                except:
                    continue
                break
            try:
                table =soup.select('table')[0]
                rows = table.find_all('tr')
                for row in rows[1:]:
                    x=row.find_all('a', href=True)
                    lst.append(row.find_all('a', href=True))
            except:
                continue
    return lst

In [6]:
esp_height=get_player_heights(1)

Spanish-ACB
[<a href="/player/Dmitry-Flis/Summary/39726">Dmitry Flis</a>]
[<a href="/player/Jon-Santamaria/Summary/39730">Jon Santamaria</a>]
[<a href="/player/Javier-Nasarre/Summary/13024">Javier Nasarre</a>]
[<a href="/player/Alejandro-Alloatti/Summary/30656">Alejandro Alloatti</a>]
[<a href="/player/Ander-Garcia/Summary/39742">Ander Garcia</a>]
[<a href="/player/Bozhidar-Avramov/Summary/25171">Bozhidar Avramov</a>]
[<a href="/player/Manuel-Rodriguez/Summary/39632">Manuel Rodriguez</a>]
[<a href="/player/Billy-Baptist/Summary/15602">Billy Baptist</a>]
[<a href="/player/Fabio-Santana/Summary/29043">Fabio Santana</a>]
[<a href="/player/Joan-Pardina/Summary/39709">Joan Pardina</a>]
[<a href="/player/Juan-Ballesta/Summary/67176">Juan Ballesta</a>]
[<a href="/player/Juan-Llorente/Summary/87684">Juan Llorente</a>]
[<a href="/player/David-Jofresa/Summary/32963">David Jofresa</a>]
[<a href="/player/Nikola-Maric/Summary/53830">Nikola Maric</a>]
[<a href="/player/Emilio-Martinez/Summary/131002

In [7]:
it_height=get_player_heights(2)

Italian-Lega-Basket-Serie-A
[<a href="/player/Lorenzo-Novati/Summary/133122">Lorenzo Novati</a>]
[<a href="/player/Cesare-Monzardo/Summary/139099">Cesare Monzardo</a>]
[<a href="/player/Alessandro-Zorzolo/Summary/130876">Alessandro Zorzolo</a>]
[<a href="/player/Giuseppe-Sindoni/Summary/121546">Giuseppe Sindoni</a>]
[<a href="/player/Kamil-Pietras/Summary/25514">Kamil Pietras</a>]
[<a href="/player/Giovanni-Rispoli/Summary/120549">Giovanni Rispoli</a>]
[<a href="/player/Giovanni-Napodano/Summary/119743">Giovanni Napodano</a>]
[<a href="/player/Alessandro-Testa/Summary/119636">Alessandro Testa</a>]
[<a href="/player/Awudu-Abass/Summary/30672">Awudu Abass</a>]
[<a href="/player/Davide-Cesana/Summary/48111">Davide Cesana</a>]
[<a href="/player/Gianmarco-Leggio/Summary/33203">Gianmarco Leggio</a>]
[<a href="/player/Riccardo-Cattapan/Summary/54306">Riccardo Cattapan</a>]
[<a href="/player/Francesco-Galli/Summary/100404">Francesco Galli</a>]
[<a href="/player/Vincenzo-Taddeo/Summary/119139">

In [10]:
tur_height=get_player_heights(3)

Turkish-BSL


In [11]:
gr_height=get_player_heights(4)

Greek-HEBA-A1


In [12]:
fr_height=get_player_heights(5)

French-Jeep-Elite


In [13]:
adr_height=get_player_heights(6)

Adriatic-League-Liga-ABA


In [14]:
rus_height=get_player_heights(7)

VTB-United-League


In [25]:
eur_lst=esp_height+it_height+tur_height+gr_height+fr_height+adr_height+rus_height

In [27]:
eur_links=[]
eur_names=[]
for i in range(len(eur_lst)):
    try:
        s=str(eur_lst[i][0])
        eur_names.append(re.findall('>([^"]*)<', s))
        eur_links.append(re.findall('"([^"]*)"', s))
    except:
        pass

In [30]:
eur_df=pd.DataFrame(eur_names,columns=['names'])

In [31]:
eur_df['link']=pd.DataFrame(eur_links,columns=['link'])

In [32]:
eur_df.drop_duplicates(inplace=True)

In [33]:
eur_df.reset_index(drop=True,inplace=True)

In [55]:
eur_height=pd.DataFrame(index=eur_df['link'],columns=['height'])
k=0
for i in eur_df['link']:
    test=0
    while test==0:
        try:
            url = 'https://basketball.realgm.com'+i
            html = urlopen(url)
            soup = BeautifulSoup(html, features="lxml")
            test=1
        except:
            continue
        break
    s=soup.find(string=re.compile("cm\)"))
    try:
        loc=s.find('cm')
        eur_height.loc[i,'height']=s[loc-3:loc]
    except:
        eur_height.loc[i,'height']='No'
    k+=1
    print(k)

193
185
201
206
206
201
193
201
198
198
203
206
201
208
198
206
193
183
203
211
191
203
206
201
201
196
188
198
208
191
213
201
180
193
203
201
191
198
206
198
188
180
211
198
203
206
208
180
201
208
188
216
193
206
203
196
206
203
206
208
206
206
203
191
201
201
193
201
206
206
183
208
201
191
196
183
216
191
188
196
218
180
185
198
201
206
196
185
201
206
201
208
196
196
201
198
180
203
208
203
208
208
191
188
196
203
193
201
203
206
193
188
211
193
188
196
188
191
196
208
216
196
191
206
196
196
198
208
193
203
185
188
201
198
211
193
196
188
185
201
198
206
198
206
196
203
196
183
206
211
191
188
196
213
203
175
203
188
201
196
196
203
193
191
206
188
191
208
206
198
201
193
196
206
203
206
198
178
198
185
206
203
211
211
203
203
198
208
218
208
185
193
185
203
203
206
211
198
213
201
193
203
193
201
201
188
201
208
196
216
203
203
196
203
196
185
203
203
198
178
203
196
183
211
191
203
218
178
191
203
203
191
206
201
203
193
208
193
188
201
196
203
206
188
203
191
191
208
206
188


196
198
196
206
198
206
185
206
211
183
203
188
196
185
188
201
203
213
183
203
211
206
201
198
206
203
203
191
203
203
188
188
193
201
185
170
226
201
203
208
191
208
201
206
203
193
196
196
203
206
206
211
206
185
185
178
208
213
201
208
203
201
196
198
193
203
206
196
201
201
188
191
196
185
203
191
201
198
196
191
198
191
196
196
216
198
196


AttributeError: 'NoneType' object has no attribute 'find'

In [158]:
eur_df.to_csv('eur_heights.csv',index=False)